In [121]:
save_sparse_csr("full_working_data",x_data)
save_sparse_csr("labels_of_full_working_data",y_data)

In [308]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [12]:
%who_ls

['Series',
 'data_dir',
 'data_set',
 'final_data',
 'ngrams',
 'np',
 'pd',
 'pickle',
 'profile_urls',
 'root_dir',
 'task_dir']

In [316]:
urls=pd.read_csv(os.path.join(final_data,"final_scraped_urls.csv"),header=None)[1]

In [1]:
from nltk import ngrams
import pandas as pd
import numpy as np 
from pandas import Series
import pickle
import os
pd.options.display.max_rows=3000
root_dir = os.path.abspath('../..')
task_dir= os.path.join(root_dir,'user_profile')
data_dir=os.path.join(task_dir,'data')
final_data=os.path.join(data_dir,"final_scrap")

In [7]:
import urlparse
def hostname(x):
    return urlparse.urlparse(x).hostname

def path(x):
    return urlparse.urlparse(x).path

def preprocessor(list_):
    return list("".join(list_))

def gram_generator(x):
    grams = ngrams(x, 3)
    return ["".join(gram) for gram in grams]

def character_extracter(x):
    return x.str.lower().str.replace("http",".").str.replace("www",".").str.findall(r"\w+").apply(preprocessor)


def joined(list_):
    return " ".join(list_)

In [174]:
from scipy.sparse import *

In [155]:
def save_sparse_csr(filename,array):
    np.savez(filename,data = array.data ,indices=array.indices,
             indptr =array.indptr, shape=array.shape )

def load_sparse_csr(filename):
    loader = np.load(filename)
    return csr_matrix((  loader['data'], loader['indices'], loader['indptr']),
                         shape = loader['shape'])

In [156]:
from sklearn import svm
from sklearn.utils import shuffle

In [12]:
"""Opens and stores the vocab for generating 3-gram features into vocab_ list for vectorizer input"""

the_filename="test_vocab"
with open(the_filename, 'rb') as f:
    vocab_ = pickle.load(f)

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizerY= CountVectorizer(min_df=1)
vectorizerY.fit_transform(vocab_)
"""generating vocab for feature extraction"""

'generating vocab for feature extraction'

In [ ]:
vectorizerY.transform(profile_urls[])

In [256]:
%reset_selective -f profile_urls
%reset_selective -f non_profile_urls
%reset_selective -f x_
%reset_selective -f y_

In [257]:
#load the saved files
profile_urls=pd.read_csv(os.path.join(final_data,"profile_urls_from_working_data.csv"),header=None)[1]
non_profile_urls=pd.read_csv(os.path.join(final_data,"non_profile_urls_from_working_data.csv"),header=None)[1]

In [16]:
special_characters=['#','/', '%', '~', '=','?','&',"-","(",")","+","@","[","]","'","*"]

In [159]:
special_=[" ".join(special_)]

In [141]:
vectorizer_spcl= CountVectorizer(min_df=1)
vectorizer_spcl.fit_transform([" ".join(special_)])

<1x16 sparse matrix of type '<type 'numpy.int64'>'
	with 16 stored elements in Compressed Sparse Row format>

In [150]:
def encoder(x):
    x=x.str.replace("http://|https://","").str.findall("[^\:\w\.]").apply(joined)
    for n,spcl in enumerate(special_characters):
        x=x.str.replace(spcl,str(n+10))
    return x

In [185]:
csc_matrix(np.reshape(profile_urls.str.len().as_matrix(),(profile_urls.shape[0],1)))

<14961x1 sparse matrix of type '<type 'numpy.int64'>'
	with 14961 stored elements in Compressed Sparse Column format>

In [294]:
x_positive_len=csc_matrix(np.reshape(profile_urls.str.len().as_matrix(),(profile_urls.shape[0],1)))
x_negative_len=csc_matrix(np.reshape(non_profile_urls.str.len().as_matrix(),(non_profile_urls.shape[0],1)))

In [295]:
x_spcl_positive=vectorizer_spcl.transform(encoder(profile_urls).values)
x_spcl_negative=vectorizer_spcl.transform(encoder(non_profile_urls).values)

In [296]:
x_positive,y_positive=complete_processor(profile_urls,type_=1)
x_negative,y_negative=complete_processor(non_profile_urls,type_=0)

In [188]:
x_positive_len.shape

(14961, 1)

In [261]:
x_pos=hstack([x_positive,x_spcl_positive,x_positive_len])
x_neg=hstack([x_negative,x_spcl_negative,x_negative_len])

In [216]:
x_pos=hstack([x_positive,x_positive_len])
x_neg=hstack([x_negative,x_negative_len])

In [297]:
x_pos=hstack([x_spcl_positive,x_positive_len])
x_neg=hstack([x_spcl_negative,x_negative_len])

In [161]:
def complete_processor(x,type_=1):
    x=x.iloc[np.random.permutation(x.shape[0])]
    x=character_extracter(x)
    x=x.apply(gram_generator).apply(joined).values
    x_=vectorizerY.transform(x)
    if type_==1:
        y_=csr_matrix(np.ones((x_.shape[0],1)))
    elif type_==0:
        y_=csr_matrix(np.zeros((x.shape[0],1)))

    return x_,y_

In [192]:
def label_formatter(y):
    return np.reshape(y.toarray(),(y.shape[0],))

In [298]:
x_data=vstack([x_neg,x_pos])
y_data=vstack([y_negative,y_positive])
x_data,y_data=shuffle(x_data,y_data)

In [299]:
x_train=x_data[:60000,:]
y_train=y_data[:60000,:]
x_val=x_data[60000:75000,:]
y_val=y_data[60000:75000,:]
x_test=x_data[75000:,:]
y_test=y_data[75000:,:]

In [300]:
clf=svm.LinearSVC(C=1.0).fit(x_train,label_formatter(y_train))

In [287]:
clf.score(x_val,label_formatter(y_val))

0.97666666666666668

In [288]:
clf.score(x_train,label_formatter(y_train))

0.9926666666666667

In [301]:
f1_score(label_formatter(y_train),clf.predict(x_train))

0.26348348871656357

In [302]:
f1_score(label_formatter(y_val),clf.predict(x_val))

0.26950842213819182

In [277]:
clf.score(x_test,label_formatter(y_test))

0.978024737866516

In [304]:
precision_score(label_formatter(y_train),clf.predict(x_train))

0.950345694531741

In [303]:
recall_score(label_formatter(y_train),clf.predict(x_train))

0.15294355654460853

In [200]:
from sklearn.metrics import *

In [305]:
import cPickle
with open('svm_with_higher_precision.pkl', 'wb') as fid:
    cPickle.dump(clf, fid)    

In [255]:
with open('feature_genrator_special_charac.pkl', 'wb') as fid:
    cPickle.dump(vectorizer_spcl,fid)

In [251]:
with open('feature_genrator.pkl', 'rb') as fid:
    vec_loaded=cPickle.load(fid)

In [235]:
import cPickle
# load it again
with open('svm_with_higher_recall.pkl', 'rb') as fid:
    clf_loaded = cPickle.load(fid)

In [8]:
data_set=pd.read_csv(os.path.join(data_dir,"data_augmented.csv")).Sources1

In [3]:
profile_urls=pd.read_csv(os.path.join(final_data,"profile_urls_from_working_data.csv"),header=None)[1]

In [9]:
data_set=data_set[np.invert(data_set.isin(profile_urls))]

In [11]:
data_set.to_csv(os.path.join(data_dir,"not_captured_in_scrapping.csv"),index_label=None,index=None)

In [13]:
profile_urls[:10]

0      http://www.math.tamu.edu/~rowell/
1      http://www.math.tamu.edu/~sdutta/
2      http://www.math.tamu.edu/~fulges/
3         http://www.math.tamu.edu/~jml/
4        http://www.math.tamu.edu/~kerr/
5       http://www.math.tamu.edu/~steve/
6    http://www.math.tamu.edu/~belmonte/
7     http://www.math.tamu.edu/~dmanuel/
8       http://www.math.tamu.edu/~fatma/
9       http://www.math.tamu.edu/~laura/
Name: 1, dtype: object